In [6]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse
import requests, re, time
import pandas_datareader
import pickle
import math

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import seaborn as sns 

import statsmodels.api as sm
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver
import time

import geopy.distance

In [7]:
### Define our Connector

import requests,os,time
def ratelimit(dt):
    "A function that handles the rate of your calls."
    time.sleep(dt) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30,waiting_time=0.5):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    self.waiting_time = waiting_time # define simple rate_limit parameter.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit(self.waiting_time)
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit(self.waiting_time)
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

# Using the connector

In [19]:
connector = Connector('MetroLog.csv',overwrite_log=True,path2selenium='/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Gecko/geckodriver',connector_type='selenium')

In [83]:
# scraping Wikipedia:
def WikiScraper(no_metros):
    url = 'https://da.wikipedia.org/wiki/Stationer_p%C3%A5_K%C3%B8benhavns_Metro'
    response,call_id = connector.get(url,'Metroer')
    time.sleep(1)

    # Creating DataFrame and dictionary for information-storing:
    data = pd.DataFrame()
    dic_list = []


    for i in range(1,no_metros+1):
        link = connector.browser.find_element_by_xpath(f'/html/body/div[3]/div[3]/div[5]/div/table[1]/tbody/tr[{i}]/td[1]/a')
        link.click()
        time.sleep(1)

        # READ INFORMATION
        # 1.1) Appending all information to DataFrame
        table = pd.read_html(connector.browser.page_source)[0]
        data[f'Traits_{i}'] = table.iloc[:,0]
        data[f'{i}'] = table.iloc[:,1]

        # 1.2) Finding extra information, that the read_html doesn't pick up:
        soup = BeautifulSoup(connector.browser.page_source, 'lxml')
        name = soup.find('h1').text
        data[f'Traits_{i}'][0] = 'Name' # Adding the name to first row of the df (This is always NaN, so doesn't delete vital data)
        data[f'{i}'][0] = name

        # 2) Creating a dictionary with all information, ensuring homogeneity between different WikiPages:
        dic = {}
        for j in range(0,len(data[f'Traits_{i}'])):
            dic[str(data[f'Traits_{i}'][j])] = str(data[f'{i}'][j])

        dic_list.append(dic)

        # RETURN 
        connector.browser.get(url)
        time.sleep(1)

    return [data,dic_list]
    #return dic_list

In [87]:
MetroScrape = WikiScraper(44)

# Working with the data

In [149]:
indexlist = ['Name','Adresse','Jernbane','Åbningsdato']
MetroList = pd.DataFrame(index=indexlist)

for i in range(0,44):
    specs = []
    for j in indexlist:
        try: 
            specs.append(MetroScrape[1][i][j]) # Appending the relevant Metro-information (j-key from scrape-dict)
        except:
            specs.append('NaN')
    # Indexing column names:
    MetroList[i] = specs
MetroList = MetroList.transpose()
MetroList.columns = ['Name', 'Address', 'Railroad', 'Opening Date'] # Changing names to english


# Cleaning for '[1]' (from Wikipedia-references):
MetroList['Address'] = [i.replace('[1]','') for i in MetroList['Address']]
MetroList['Address'] = [i.replace('[2]','') for i in MetroList['Address']]
MetroList['Railroad'] = [i.replace('[2]','') for i in MetroList['Railroad']]
MetroList['Opening Date'] = [i.replace('[1]','') for i in MetroList['Opening Date']]

# Cleaning NaN-values:
MetroList = MetroList.drop(MetroList["Address"].loc[MetroList["Address"]=='NaN'].index)
MetroList.reset_index(inplace=True)
del MetroList['index']

# Creating dummy-variable for Cityring:
MetroList['Cityring'] = 0
for i in range(0,len(MetroList)):
    if 'Cityringen' in MetroList['Railroad'][i]:
        MetroList['Cityring'][i] = 1

# Simplyfying Opening Date-column to year for use in regression analysis later:
for i in range(0,len(MetroList)):
    try:
        year = int(MetroList['Opening Date'][i][-4:])
        if 0 < year < 2021:
            MetroList['Opening Date'][i] = int(MetroList['Opening Date'][i][-4:])
    except:
        pass

# Cleaning the rest by hand, example: '13. december 1986 (S-tog), 12. oktober 2003 (Metro)' is cleaned to 2003
for i in [7,32]:
    MetroList['Opening Date'][i] = 2003
MetroList['Opening Date'][25] = 2002
for i in [23,27,31,34]:
    MetroList['Opening Date'][i] = 2019
MetroList = MetroList.drop(22)
MetroList.reset_index(inplace=True)
del MetroList['index']

In [151]:
MetroList.head(n=3)

,Name,Address,Railroad,Opening Date,Cityring
0,Aksel Møllers Have Station,Godthåbsvej 312000 Frederiksberg,Cityringen,2019,1
1,Amager Strand Station,Italiensvej 72A2300 København S,Østamagerbanen,2007,0
2,Amagerbro Station,Amagerbro Torv 12300 København S,Ørestadsbanen,2002,0


Seeing that the coordinates returned equals (39.78373,-100.445882) if there's a mistake (mostly, that the door and zip code are not separated by space) in the Address, I fix the very last issues manually in Excel in about a minute. This is no problem, as I am only working with 44 entries, and it would have been relatively difficult using RegEx, as the apartment numbers do not have the same number of digits and sometimes have letters in them. Too heterogenous. Speed before unneeded and superfluously complex code!

In [152]:
# Saving to Excel:
MetroList.to_excel('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metro Adresses for Final Touches.xlsx')

In [153]:
# Pulling edited version:
MetroList = pd.read_excel('/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Pickles/Metro Adresses for Final Touches_DONE.xlsx')

In [155]:
del MetroList['Unnamed: 0']
MetroList.head(n=3)

,Name,Address,Railroad,Opening Date,Cityring
0,Aksel Møllers Have Station,Godthåbsvej 31 2000 Frederiksberg,Cityringen,2019,1
1,Amager Strand Station,Italiensvej 72A 2300 København S,Østamagerbanen,2007,0
2,Amagerbro Station,Amagerbro Torv 1 2300 København S,Ørestadsbanen,2002,0
